In [1]:
! pip install opendatasets --quiet

In [2]:
import opendatasets as od
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
#{"username":"rahaf8","key":"f59b8cb26f2973bc6fb4c52b1516ac19"}

dataset_url = 'https://www.kaggle.com/datasets/shashiprakash204/ucfcrimeminidataset'

# Specify the folder or file you want to download

od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: rahaf8
Your Kaggle Key: ··········


100%|██████████| 897M/897M [00:09<00:00, 99.0MB/s]


In [5]:
dataset_dir = "/content/ucfcrimeminidataset/dataset"


In [6]:
labels = ['Abuse','Arrest','Assault','Burglary','Fighting',"Normal"]
